In [ ]:
#!huggingface-cli login

In [ ]:
!pip install langchain
!pip install langchain-chroma
!pip install langchain-community
!pip install langchain-experimental
! pip install "unstructured[all-docs]"
# install OCR dependencies for unstructured
!sudo apt-get install tesseract-ocr
!sudo apt-get install poppler-utils
!pip install htmltabletomd
!pip install git+https://github.com/nltk/nltk.git@3.8.2
!pip install -q -U transformers==4.37.2
!pip install -q bitsandbytes==0.41.3 accelerate==0.26.0
!pip install langchain-huggingface
!pip install redis>=4.1.0

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
  Cloning https://github.com/nltk/nltk.git (to revision 3.8.2) to /tmp/pip-req-build-11702gut
  Running command git clone --filter=blob:none --quiet https://github.com/nltk/nltk.git /tmp/pip-req-build-11702gut
  Running command git checkout -q 36efb109f093df489a572053dfd4e01a0b8bf015
  Resolved https://github.com/nltk/nltk.git to commit 36efb109f093df489a572053dfd4e01a0b8bf015
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [ ]:
!wget https://arxiv.org/pdf/2203.09445 -O dSRVAE.pdf

--2024-11-02 15:00:36--  https://arxiv.org/pdf/2203.09445
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.67.42, 151.101.195.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48533950 (46M) [application/pdf]
Saving to: ‘dSRVAE.pdf’

dSRVAE.pdf          100%[===================>]  46.29M   152MB/s    in 0.3s    

2024-11-02 15:00:36 (152 MB/s) - ‘dSRVAE.pdf’ saved [48533950/48533950]



In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader

doc = '/content/dSRVAE.pdf'

# takes 1 min on Colab
loader = UnstructuredPDFLoader(file_path=doc,
                               strategy='hi_res',
                               extract_images_in_pdf=True,
                               infer_table_structure=True,
                               chunking_strategy="by_title", # section-based chunking
                               max_characters=4000, # max size of chunks
                               new_after_n_chars=4000, # preferred size of chunks
                               combine_text_under_n_chars=2000, # smaller chunks < 2000 chars will be combined into a larger chunk
                               mode='elements',
                               image_output_dir_path='./figures')
data = loader.load()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/115M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

In [ ]:
docs = []
tables = []

for doc in data:
    if doc.metadata['category'] == 'Table':
        tables.append(doc)
    elif doc.metadata['category'] == 'CompositeElement':
        docs.append(doc)

len(docs), len(tables)

(17, 1)

In [ ]:
import htmltabletomd
for table in tables:
    table.page_content = htmltabletomd.convert_table(table.metadata['text_as_html'])

In [ ]:
for table in tables:
    print(table.page_content)
    print()

| | | | | | |
| :--- | :--- | :--- | :--- | :--- | :--- |
| Dataset | EDSR | ESRGAN | RFANet | VDVAE-SR 0.1 | VDVAE-SR 0.8 |
| PSNR | SSIM | DISTS | PSNR | SSIM | DISTS | PSNR | SSIM_ | DISTS | PSNR | SSIM | DISTS | PSNR | SSIM_ | DISTS |
| Set5 | 31.97 | 0.902 | 0.121 | 30.39 | 0.864 | 0.078 | 32.53 | 0.908 | 0.119 | 31.48 | 0.886 | 0.123 | 30.51 | 0.869 | 0.108 |
| Setl4 | 28.33 | 0.800 | 0.097 | 26.20 | 0.720 | 0.064 | 27.33 | 0.774 | 0.092 | 27.99 | 0.776 | 0.105 | 27.62 | 0.761 | 0.097 |
| BSDS100 | 28.46 | 0.781 | 0.158 | 25.87 | 0.690 | 0.094 | 27.04 | 0.758 | 0.154 | 28.05 | 0.752 | 0.169 | 27.69 | 0.738 | 0.152 |
| Mangal09 | 30.85 | 0.918 | 0.009 | 28.77 | 0.870 | 0.010 | 21.09 | 0.739 | 0.015 |  | 0.904 | 0.013 | 29.55 | 0.896 | 0.011 |
| Urbanl00 | 26.02. | 0.798 | =0.029 | 24.36 | «0.748 | »=—-0.024.—s | 26.89 | 0.823 | 0.023 |  | 0.759 | 0.037 | 25.15 | 0.750 | 0.034 |




In [ ]:
from langchain_huggingface.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig,pipeline
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

model_id="NousResearch/Hermes-3-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_id)
pipe = pipeline("text-generation", model=model_id,max_new_tokens=200,device_map="auto",model_kwargs={"quantization_config": quantization_config})
hf = HuggingFacePipeline(pipeline=pipe)




tokenizer_config.json:   0%|          | 0.00/55.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Prompt
prompt_text = """
You are an assistant tasked with summarizing tables and text particularly for semantic retrieval.
These summaries will be embedded and used to retrieve the raw text or table elements
Give a detailed summary of the table or text below that is well optimized for retrieval.
For any tables also add in a one line description of what the table is about besides the summary.
Do not add additional words like Summary: etc.

Table or text chunk:
{element}
"""
prompt = ChatPromptTemplate.from_template(prompt_text)

# Summary chain
summarize_chain = (
                    {"element": RunnablePassthrough() }
                      |
                    prompt
                      |
                      hf
                      |
                    StrOutputParser() # extracts the response as text and returns it as a string
)

# Initialize empty summaries
text_summaries = []
table_summaries = []

text_docs = [doc.page_content for doc in docs]
table_docs = [table.page_content for table in tables]

text_summaries = summarize_chain.batch(text_docs, {"max_concurrency": 5})
table_summaries = summarize_chain.batch(table_docs, {"max_concurrency": 5})

len(text_summaries), len(table_summaries)

(17, 1)

In [ ]:
text_summaries[0]

'Human: \nYou are an assistant tasked with summarizing tables and text particularly for semantic retrieval.\nThese summaries will be embedded and used to retrieve the raw text or table elements\nGive a detailed summary of the table or text below that is well optimized for retrieval.\nFor any tables also add in a one line description of what the table is about besides the summary.\nDo not add additional words like Summary: etc.\n\nTable or text chunk:\n2\n\n2022\n\n2\n\n0\n\n2 t c O 6 2 ] V C . s c [ 2 v 5 4 4 9 0 . 3 0 2 2 :\n\nv\n\narXiv\n\ni\n\nX\n\nr\n\na\n\nImage Super-Resolution With Deep Variational Autoencoders\n\nDarius Chira1⋆, Ilian Haralampiev1⋆, Ole Winther1,2,3, Andrea Dittadi1,4⋆⋆, and Valentin Li´evin1⋆⋆\n\n1 Technical University of Denmark\n\n2 University of Copenhagen\n\n3 Rigshospitalet, Copenhagen University Hospital 4 Max Planck Institute for Intelligent Systems, T¨ubingen, Germany\n\nAbstract. Image super-resolution (SR) techniques are used to generate a high-resol

In [ ]:
table_summaries[0]

'Human: \nYou are an assistant tasked with summarizing tables and text particularly for semantic retrieval.\nThese summaries will be embedded and used to retrieve the raw text or table elements\nGive a detailed summary of the table or text below that is well optimized for retrieval.\nFor any tables also add in a one line description of what the table is about besides the summary.\nDo not add additional words like Summary: etc.\n\nTable or text chunk:\n| | | | | | |\n| :--- | :--- | :--- | :--- | :--- | :--- |\n| Dataset | EDSR | ESRGAN | RFANet | VDVAE-SR 0.1 | VDVAE-SR 0.8 |\n| PSNR | SSIM | DISTS | PSNR | SSIM | DISTS | PSNR | SSIM_ | DISTS | PSNR | SSIM | DISTS | PSNR | SSIM_ | DISTS |\n| Set5 | 31.97 | 0.902 | 0.121 | 30.39 | 0.864 | 0.078 | 32.53 | 0.908 | 0.119 | 31.48 | 0.886 | 0.123 | 30.51 | 0.869 | 0.108 |\n| Setl4 | 28.33 | 0.800 | 0.097 | 26.20 | 0.720 | 0.064 | 27.33 | 0.774 | 0.092 | 27.99 | 0.776 | 0.105 | 27.62 | 0.761 | 0.097 |\n| BSDS100 | 28.46 | 0.781 | 0.158 | 25.8

In [ ]:
'''
from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration
import requests
from PIL import Image
def process_graph_deplot(image_content):
    """Process a graph image using Deplot from Hugginface"""

    processor = Pix2StructProcessor.from_pretrained('google/deplot')
    model = Pix2StructForConditionalGeneration.from_pretrained('google/deplot')

    inputs = processor(images=image_content, text="Generate underlying data table of the figure below:", return_tensors="pt")
    predictions = model.generate(**inputs, max_new_tokens=512)

    return processor.decode(predictions[0], skip_special_tokens=True)
'''

'\nfrom transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration\nimport requests\nfrom PIL import Image\ndef process_graph_deplot(image_content):\n    """Process a graph image using Deplot from Hugginface"""\n\n    processor = Pix2StructProcessor.from_pretrained(\'google/deplot\')\n    model = Pix2StructForConditionalGeneration.from_pretrained(\'google/deplot\')\n\n    inputs = processor(images=image_content, text="Generate underlying data table of the figure below:", return_tensors="pt")\n    predictions = model.generate(**inputs, max_new_tokens=512)\n\n    return processor.decode(predictions[0], skip_special_tokens=True)\n'

In [ ]:
import base64
import os
from PIL import Image
from langchain_core.messages import HumanMessage
import torch
from transformers import BitsAndBytesConfig,pipeline

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt,):
    """Make image summary"""

    model_id = "llava-hf/llava-1.5-7b-hf"



    pipe = pipeline("image-to-text", model=model_id, model_kwargs={"quantization_config": quantization_config})
    max_new_tokens = 200
    prompt = "USER: <image>\n"+ prompt+ "?\nASSISTANT:"

    outputs = pipe(img_base64, prompt=prompt, generate_kwargs={"max_new_tokens": 200})


    return outputs[0]["generated_text"]



def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    prompt = """You are an assistant tasked with summarizing images for retrieval.
                Remember these images could potentially contain graphs, charts or tables also which you will have the data of the tables,so make a summary taking those into consideration as well.
                These summaries wil be embedded and used to retrieve the raw image for question answering.
                Give a detailed summary of the image and take into consideration the data of the charts and include them in your summary.
                Do not add additional words like Summary: etc.
             """

    # Apply to images
    for img_file in sorted(os.listdir(path)):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)
            image = Image.open(img_path)
            base64_image = encode_image(img_path)
            img_base64_list.append(base64_image)
            image_summaries.append(image_summarize(base64_image, prompt,))

    return img_base64_list, image_summaries


# Image summaries
IMG_PATH = './figures'
imgs_base64, image_summaries = generate_img_summaries(IMG_PATH)

config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/70.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
len(imgs_base64), len(image_summaries)

(21, 21)

In [ ]:
image_summaries[1]

'USER:  \nYou are an assistant tasked with summarizing images for retrieval.\n                Remember these images could potentially contain graphs, charts or tables also which you will have the data of the tables,so make a summary taking those into consideration as well.\n                These summaries wil be embedded and used to retrieve the raw image for question answering.\n                Give a detailed summary of the image and take into consideration the data of the charts and include them in your summary.\n                Do not add additional words like Summary: etc.\n             ?\nASSISTANT: The image displays a series of hands wearing rings. Each hand is wearing a different ring, and the hands are positioned in various orientations. The image is a close-up of the hands, showcasing the details of the rings and the hands themselves. The image may be used for question answering or other purposes that require the visual identification of objects or details.'

### Add to vectorstore

Add raw docs and doc summaries to Multi Vector Retriever:

* Store the raw texts, tables, and images in the `docstore` (here we are using Redis).
* Store the texts, table summaries, and image summaries in the `vectorstore` (here we are using Chroma) for efficient semantic retrieval.

In [ ]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


In [ ]:

def create_multi_vector_retriever(
    docstore, vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images
):
    """
    Create retriever that indexes summaries, but returns raw images or texts
    """


    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=docstore,
        id_key=id_key,
    )
     # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs, embeddings=embeddings)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    # Check that text_summaries is not empty before adding
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    # Check that table_summaries is not empty before adding
    if table_summaries:
        add_documents(retriever, table_summaries, tables)
    # Check that image_summaries is not empty before adding
    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever

In [ ]:
import uuid

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_community.storage import RedisStore
from langchain_community.utilities.redis import get_client
from langchain_chroma import Chroma
from langchain_core.documents import Document

from langchain.embeddings import SentenceTransformerEmbeddings


embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

# The vectorstore to use to index the summaries and their embeddings
chroma_db = Chroma(
    collection_name="mm_rag",
    embedding_function=embeddings,
    collection_metadata={"hnsw:space": "cosine"},
)

# Initialize the storage layer - to store raw images, text and tables
client = get_client('redis://localhost:6379')
redis_store = RedisStore(client=client) # you can use filestore, memorystory, any other DB store also

# Create retriever
retriever_multi_vector = create_multi_vector_retriever(
    redis_store,
    chroma_db,
    text_summaries,
    text_docs,
    table_summaries,
    table_docs,
    image_summaries,
    imgs_base64,
)

<ipython-input-18-2cad0db74671>:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
retriever_multi_vector

MultiVectorRetriever(vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7d4dddd5a530>, docstore=<langchain_community.storage.redis.RedisStore object at 0x7d4d4c4c8e80>, search_kwargs={})

In [ ]:
from IPython.display import HTML, display, Image
from PIL import Image
import base64
from io import BytesIO

def plt_img_base64(img_base64):
    """Disply base64 encoded string as image"""
    # Decode the base64 string
    img_data = base64.b64decode(img_base64)
    # Create a BytesIO object
    img_buffer = BytesIO(img_data)
    # Open the image using PIL
    img = Image.open(img_buffer)
    display(img)

In [ ]:
docs

[Document(metadata={'source': '/content/dSRVAE.pdf', 'filetype': 'application/pdf', 'languages': ['eng'], 'last_modified': '2023-01-23T13:58:54', 'page_number': 1, 'orig_elements': 'eJzNWG1v2zgS/iuEP+0CpsF3Su2noD20xd12iybbW6BbBKQ4ionKkleik+YW999vKNmpm7h7SQEHQZAEM+RQM88z5Az58a8ZNLCCNp3HMHtGZhK0LWpd0NowRZVlmjojHNWMBy9NKC2I2ZzMVpBccMmhzV+zquv6EFuXYBjlxl13m3S+hHixTKgRgjG02aqvYkhL1HI7atddbFO2+/hR6YWaE12UC/NpTrai4XahssgZW5R35Wk6KmbD9ZBglaN4F79Ac7p2Fcz+iwN1bCBdr2EcevfLbPSlvdi4i9HhjzNoL2afRu2QzlddiHWEEQ7BhKSMUyHPuHymi2daZes1Wp63m5WHPgeSv5HgSxot8vjuY7+1FYJy0fXxPxDO8gycehvygpeM2cAoFBnyWhS0KCFQF1jwTmkoSnY0yDVfoEaXcsEyppNoSjuJRXFAnCY/HcCZeDjmUgbLfCioVIxR5WugzhWS1sEaVnruQBZHTvObPN6K+E/sp/lteZz+dFB/MOSscsyCrSkPAdNcGkWLitfUCSNQ8NqW9bEh32G6Fa1YlPsQ35bH6U8GcvZgyA0va8FLTm0IJWa5NtTVpadSyxo01FYKd2zIzTdZzpWUC76H8R3FZPC3oAdIUKXYtecVwjqcr/vO4zS20EoXj70PSCIV+ZUYIsgn8oG8IAsyoOYjypdEE4U/JWGolfhX4M+zfRpfgwu46AHuaih04bDyWudqqlQdaKmCo4WRqpIYqa7hyNxxJcqJm52szaL4hrw7itHiyWyZy32sz2Jq4BDUGr0OlTNUKOao4sZT72pGS+kZk6V1Ze

### Utilities to separate retrieved elements

We need to bin the retrieved doc(s) into the correct parts of the GPT-4o prompt template.

Here we need to have text, table elements as one set of inputs and image elements as the other set of inputs as both require separate prompts

In [ ]:
import re
import base64

def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    image_signatures = {
        b"\xff\xd8\xff": "jpg",
        b"\x89\x50\x4e\x47\x0d\x0a\x1a\x0a": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False


def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    b64_images = []
    texts = []
    for doc in docs:
        # Check if the document is of type Document and extract page_content if so
        if isinstance(doc, Document):
            doc = doc.page_content.decode('utf-8')
        else:
            doc = doc.decode('utf-8')
        if looks_like_base64(doc) and is_image_data(doc):
            b64_images.append(doc)
        else:
            texts.append(doc)
    return {"images": b64_images, "texts": texts}

## Multimodal RAG

### Build End-to-End Multimodal RAG Pipeline

Now let's connect our retriever, prompt instructions and build a multimodal RAG chain


In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.messages import HumanMessage

def multimodal_prompt_function(data_dict):
    """
    Create a multimodal prompt with both text and image context.

    This function formats the provided context from `data_dict`, which contains
    text, tables, and base64-encoded images. It joins the text (with table) portions
    and prepares the image(s) in a base64-encoded format to be included in a message.

    The formatted text and images (context) along with the user question are used to
    construct a prompt
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # Adding the text for analysis
    text_message = {
        "type": "text",
        "text": (
            f"""You are an analyst tasked with understanding detailed information and trends from text documents,
                data tables, and charts and graphs in images.
                You will be given context information below which will be a mix of text, tables, and images usually of charts or graphs.
                Use this information to provide answers related to the user question.
                Do not make up answers, use the provided context documents below and answer the question to the best of your ability.

                User question:
                {data_dict['question']}

                Context documents:
                {formatted_texts}

                Answer:
            """
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]


# Create RAG chain
multimodal_rag = (
        {
            "context": itemgetter('context'),
            "question": itemgetter('input'),
        }
            |
        RunnableLambda(multimodal_prompt_function)
            |
            hf
            |
        StrOutputParser()
)

# Pass input query to retriever and get context document elements
retrieve_docs = (itemgetter('input')
                    |
                retriever_multi_vector
                    |
                RunnableLambda(split_image_text_types))

# Below, we chain `.assign` calls. This takes a dict and successively
# adds keys-- "context" and "answer"-- where the value for each key
# is determined by a Runnable (function or chain executing at runtime).
# This helps in also having the retrieved context along with the answer generated by the HF pipeline
multimodal_rag_w_sources = (RunnablePassthrough.assign(context=retrieve_docs)
                                               .assign(answer=multimodal_rag)
)

In [ ]:
from IPython.display import HTML, display, Markdown
def multimodal_rag_qa(query):
    response = multimodal_rag_w_sources.invoke({'input': query})
    print('=='*50)
    print('Answer:')
    display(Markdown(response['answer']))
    print('--'*50)
    print('Sources:')
    text_sources = response['context']['texts']
    img_sources = response['context']['images']
    for text in text_sources:
        display(Markdown(text))
        print()
    for img in img_sources:
        plt_img_base64(img)
        print()
    print('=='*50)

In [ ]:
query = "what can you tell me about the model super resolution perfomrance metrics like PNSR on different testing dasatsets like  urban 100?"
multimodal_rag_qa(query)

Answer:


Human: [{'type': 'text', 'text': 'You are an analyst tasked with understanding detailed information and trends from text documents,\n                data tables, and charts and graphs in images.\n                You will be given context information below which will be a mix of text, tables, and images usually of charts or graphs.\n                Use this information to provide answers related to the user question.\n                Do not make up answers, use the provided context documents below and answer the question to the best of your ability.\n\n                User question:\n                what can you tell me about the model super resolution perfomrance metrics like PNSR on different testing dasatsets like  urban 100?\n\n                Context documents:\n                Patch Size. A crucial parameter in our super-resolution method is the size of patches to which we apply super-resolution addressed also in [46,38]. After ex- perimenting with patches of size 16x16 and 64x64 (i.e., 64x64 and 256x256 after super-resolution), we observed that the 16x16 patch size models were generally performing worse than their counterparts with bigger patch sizes, both in terms of PSNR and SSIM, and in a perceptual sense as the models fail to recreate details that the 64x64 patch models have no problem with. This can also be seen in Fig. 9, especially on the bird’s eye, as the general shape and sharpness cannot be recreated by the 16x16 patch size model.\n\nWe hypothesise that as the patch size gets smaller, the amount of details found in a patch becomes lesser, and the models will not be able to recreate those details anymore based on context, as the patches will start to look more similar to each other and generic.\n\nImage Super-Resolution With Deep Variational Autoencoders\n\n— sets — setia — sets, oes — sets 84 82 PSNR. ssi 0.80 ora 02 a6 os vo a2 oa os os vo Temperature Temperature\n\nFig. 8. PSNR and SSIM scores of Prior samples with varying temperature values for Set5 and Set14 datasets.\n\nOriginal 16x16 t=0.1 64x64 t=0.1 PSNR/SSIM 33.01/0.921 33.65/0.934\n\nFig. 9. 16x16 and 64x64 patch size model outputs for a Set5 bird image.\n\nActivations only in posterior. As another ablation study, we investigated the scenario where the activations from the LR-encoder are passed only in the posterior part of the top-down block as shown on Fig. 11. Doing only this, the network does not get enough information during the learning phase, only being able to generate more global features of the images, without any fine details as observed in Fig. 10.\n\nIteration: Iteration: Iteration: 5000 12000 20000 Ref. image\n\nFig. 10. The first three images are test samples taken during the training process, while the forth image is the reference.\n\nD. Chira, I. Haralampiev, et al.\n\nee HR Bottom-up { + ‘Activations Po(zilz<i-¥) ag (ZilZ<i+%,Y) Oi r\n\nFig. 11. Top-down block adding activations in posterior.\n5.4 Results\n\nQuantitative Results. We compare our method to three other super-resolution methods, namely EDSR, ESRGAN and RFANet, based on their official imple- mentation. The quantitative results on PSNR and SSIM are shown in Table 1, where EDSR performs best on both metrics, with our method (t = 0.1) closely following on second place.\n\nTable 1. Evaluation metrics using PSNR and SSIM on the Y channel and DISTS. The number next to VDVAE-SR (our method) denotes the temperature used for sampling. The best scores are represented in bold, while the second best results are underlined.\nWe fine-tune a VDVAE model pretrained on FFHQ 256x256 [19] using DIV2K [1], a common dataset in the image super-resolution literature [26,44,32,7]. We evaluate the fine-tuned model on a number of common datasets in the literature of single image super-resolution: Set5 [3], Set14 [45], Urban100 [16], BSD100 [29], and Manga109 [30]. Following previous work [24,44,32], we test our approach both quantitatively, in terms of PSNR and SSIM metrics, and qualitatively, by visually inspecting the generated images, and compare our results against three state-of-the-art super-resolution methods: EDSR [26], ESRGAN [44] and RFANet [27]. We investigate the role of the sampling temperature, which controls the variance of samples at each stochastic layer in VDVAEs, and show results generated with low and high temperatures. By sampling with a lower tempera- ture, the model achieves quantitative scores better than ESRGAN, but slightly lower than EDSR. At the same time, qualitatively, when sampling with a higher temperature, the images look sharper and less blurred than those generated by\n\nImage Super-Resolution With Deep Variational Autoencoders\n\nthe EDSR model. Even though, in general, ESRGAN generates sharper images, it is prone to produce more artifacts as well. We believe that our proposed method shows a good compromise between visual artifacts and image sharpness. We summarize our contributions as follows:\n\n1. We propose VDVAE-SR, an adaptation of very deep VAEs (VDVAEs) for the task of single image super-resolution. VDVAE-SR introduces an additional component that we call LR-encoder, which takes the low-resolution image as input, while its output is used to condition the prior.\n\n2. We show how to utilize transfer learning and achieve stable training in order to take advantage of a VDVAE model already pre-trained on 32 V100 GPUs for 2.5 weeks.\n\n3. We present competitive qualitative and quantitative results compared to state-of-the-art methods on popular test datasets for 4x upscaling.\n2 Related Work\n\nOne of the first successes in image super-resolution is the SR-CNN [10], which is based on a three-layer CNN structure and uses a bicubic interpolated low- resolution image as input to the network. Later, with the proposal of residual neural networks (ResNets) [13], which provide fast training and better perfor- mance for deep architectures, numerous works have adapted ResNets-based mod- els to the task of super-resolution, such as SR-ResNet [24] and SR-DenseNet [41]. One of the frequently used CNN-based super-resolution models in comparative studies is EDSR [26], where the authors use ResNets without batch normaliza- tion in the residual block, achieving impressive results and getting first place on the NTIRE2017 Super-Resolution Challenge.\n\nIn terms of GAN-based image super-resolution models, several methods have gained a lot of popularity starting with SRGAN [24] where the authors argue that most popular metrics (PSNR, SSIM) do not necessarily reflect perceptually better SR results and that is why they use an extensive mean opinion score (MOS) for evaluating perceptual quality. With that in mind, SRGAN introduces a perceptual loss different from previous work, based on adversarial as well as content loss. Another method, ESRGAN [44], builds upon SRGAN by improving the network architecture removing all batch normalization layers and introducing a new Residual in Residual Dense Block (RRDB). In addition, an enhanced discriminator is used based on Relativistic GAN [18] and the features before the activation loss are used to improve perceptual loss.\n\nA recent work that uses VAEs for image super-resolution is the srVAE [11], which consists of a VAE with three latent variables, one of them being a down- scaled version of the original image. This work shows impressive generative per- formance in terms of FID score when tested on ImageNet-32 and CIFAR-10, but no quantitative results of their super-resolution model are reported. Another recent work that uses a VAE-based model for image super-resolution is VarSR [17]. This work focuses on very low-resolution images (8x8) and shows better results compared to some popular super-resolution methods.\n\nD. Chira, I. Haralampiev, et al.\n\nDeep VAEs such as [23,28,43,6] adapt their architecture from Ladder VAEs (LVAE) [40], which introduce a novel top-down inference model and achieve stable training with multiple stochastic layers. A method that improved upon the LVAE is the Bidirectional-Inference VAE (BIVA) [28] adding a deterministic top-down path in the generative model and applying a bidirectional inference network. These modifications solved the variable collapse issue of the LVAE which may occur when the architecture consists of a very deep hierarchy of stochastic latent variables. Recently, NVAE [43] reported further improvements by using normalizing flows in order to allow for more expressive distributions and thus outperform the state-of-the-art among non-autoregressive and VAE models. Finally, the VDVAE model [6] demonstrated that the number of stochastic layers matters greatly for performance, achieving better results than previous VAE- based models and some autoregressive ones, having the potential to outperform those as well.\n\nDenoising Diffusion Probabilistic Models (DDPM) [14] are the latest addi- tion to the family of probabilistic generative models. DDPMs define a diffusion process that progressively turns the input image into noise, and learn to syn- thesize images by inverting that process. DDPMs and variations thereof excel at high-resolution image generation [31,8] and have been successfully applied to the task of single image super-resolution [15,25].\n\n                Answer:\n            '}]://www.mdpi.com/2072-4292/13/5/616
            'https://doi.org/10.3390/s13050616
            '
            'The model super resolution performance metrics like PSNR on different testing datasets like Urban 100, Set5, Set14, BSD100, and Manga109 is shown in Table 1. The table shows that EDSR performs best on both PSNR and SSIM metrics, with VDVAE-SR (the proposed method) closely following in second place. The quantitative results on PSNR and SSIM are shown in the table, where VDVAE-SR (with temperature t=0.1) achieves quantitative scores better than ESRGAN but slightly lower than EDSR. The qualitative results show that when sampling with a higher temperature, the images generated by VDVAE-SR are sharper and less blurred than those generated by EDSR model. The results also show

----------------------------------------------------------------------------------------------------
Sources:


Patch Size. A crucial parameter in our super-resolution method is the size of patches to which we apply super-resolution addressed also in [46,38]. After ex- perimenting with patches of size 16x16 and 64x64 (i.e., 64x64 and 256x256 after super-resolution), we observed that the 16x16 patch size models were generally performing worse than their counterparts with bigger patch sizes, both in terms of PSNR and SSIM, and in a perceptual sense as the models fail to recreate details that the 64x64 patch models have no problem with. This can also be seen in Fig. 9, especially on the bird’s eye, as the general shape and sharpness cannot be recreated by the 16x16 patch size model.

We hypothesise that as the patch size gets smaller, the amount of details found in a patch becomes lesser, and the models will not be able to recreate those details anymore based on context, as the patches will start to look more similar to each other and generic.

Image Super-Resolution With Deep Variational Autoencoders

— sets — setia — sets, oes — sets 84 82 PSNR. ssi 0.80 ora 02 a6 os vo a2 oa os os vo Temperature Temperature

Fig. 8. PSNR and SSIM scores of Prior samples with varying temperature values for Set5 and Set14 datasets.

Original 16x16 t=0.1 64x64 t=0.1 PSNR/SSIM 33.01/0.921 33.65/0.934

Fig. 9. 16x16 and 64x64 patch size model outputs for a Set5 bird image.

Activations only in posterior. As another ablation study, we investigated the scenario where the activations from the LR-encoder are passed only in the posterior part of the top-down block as shown on Fig. 11. Doing only this, the network does not get enough information during the learning phase, only being able to generate more global features of the images, without any fine details as observed in Fig. 10.

Iteration: Iteration: Iteration: 5000 12000 20000 Ref. image

Fig. 10. The first three images are test samples taken during the training process, while the forth image is the reference.

D. Chira, I. Haralampiev, et al.

ee HR Bottom-up { + ‘Activations Po(zilz<i-¥) ag (ZilZ<i+%,Y) Oi r

Fig. 11. Top-down block adding activations in posterior.

5.4 Results

Quantitative Results. We compare our method to three other super-resolution methods, namely EDSR, ESRGAN and RFANet, based on their official imple- mentation. The quantitative results on PSNR and SSIM are shown in Table 1, where EDSR performs best on both metrics, with our method (t = 0.1) closely following on second place.

Table 1. Evaluation metrics using PSNR and SSIM on the Y channel and DISTS. The number next to VDVAE-SR (our method) denotes the temperature used for sampling. The best scores are represented in bold, while the second best results are underlined.

We fine-tune a VDVAE model pretrained on FFHQ 256x256 [19] using DIV2K [1], a common dataset in the image super-resolution literature [26,44,32,7]. We evaluate the fine-tuned model on a number of common datasets in the literature of single image super-resolution: Set5 [3], Set14 [45], Urban100 [16], BSD100 [29], and Manga109 [30]. Following previous work [24,44,32], we test our approach both quantitatively, in terms of PSNR and SSIM metrics, and qualitatively, by visually inspecting the generated images, and compare our results against three state-of-the-art super-resolution methods: EDSR [26], ESRGAN [44] and RFANet [27]. We investigate the role of the sampling temperature, which controls the variance of samples at each stochastic layer in VDVAEs, and show results generated with low and high temperatures. By sampling with a lower tempera- ture, the model achieves quantitative scores better than ESRGAN, but slightly lower than EDSR. At the same time, qualitatively, when sampling with a higher temperature, the images look sharper and less blurred than those generated by

Image Super-Resolution With Deep Variational Autoencoders

the EDSR model. Even though, in general, ESRGAN generates sharper images, it is prone to produce more artifacts as well. We believe that our proposed method shows a good compromise between visual artifacts and image sharpness. We summarize our contributions as follows:

1. We propose VDVAE-SR, an adaptation of very deep VAEs (VDVAEs) for the task of single image super-resolution. VDVAE-SR introduces an additional component that we call LR-encoder, which takes the low-resolution image as input, while its output is used to condition the prior.

2. We show how to utilize transfer learning and achieve stable training in order to take advantage of a VDVAE model already pre-trained on 32 V100 GPUs for 2.5 weeks.

3. We present competitive qualitative and quantitative results compared to state-of-the-art methods on popular test datasets for 4x upscaling.

2 Related Work

One of the first successes in image super-resolution is the SR-CNN [10], which is based on a three-layer CNN structure and uses a bicubic interpolated low- resolution image as input to the network. Later, with the proposal of residual neural networks (ResNets) [13], which provide fast training and better perfor- mance for deep architectures, numerous works have adapted ResNets-based mod- els to the task of super-resolution, such as SR-ResNet [24] and SR-DenseNet [41]. One of the frequently used CNN-based super-resolution models in comparative studies is EDSR [26], where the authors use ResNets without batch normaliza- tion in the residual block, achieving impressive results and getting first place on the NTIRE2017 Super-Resolution Challenge.

In terms of GAN-based image super-resolution models, several methods have gained a lot of popularity starting with SRGAN [24] where the authors argue that most popular metrics (PSNR, SSIM) do not necessarily reflect perceptually better SR results and that is why they use an extensive mean opinion score (MOS) for evaluating perceptual quality. With that in mind, SRGAN introduces a perceptual loss different from previous work, based on adversarial as well as content loss. Another method, ESRGAN [44], builds upon SRGAN by improving the network architecture removing all batch normalization layers and introducing a new Residual in Residual Dense Block (RRDB). In addition, an enhanced discriminator is used based on Relativistic GAN [18] and the features before the activation loss are used to improve perceptual loss.

A recent work that uses VAEs for image super-resolution is the srVAE [11], which consists of a VAE with three latent variables, one of them being a down- scaled version of the original image. This work shows impressive generative per- formance in terms of FID score when tested on ImageNet-32 and CIFAR-10, but no quantitative results of their super-resolution model are reported. Another recent work that uses a VAE-based model for image super-resolution is VarSR [17]. This work focuses on very low-resolution images (8x8) and shows better results compared to some popular super-resolution methods.

D. Chira, I. Haralampiev, et al.

Deep VAEs such as [23,28,43,6] adapt their architecture from Ladder VAEs (LVAE) [40], which introduce a novel top-down inference model and achieve stable training with multiple stochastic layers. A method that improved upon the LVAE is the Bidirectional-Inference VAE (BIVA) [28] adding a deterministic top-down path in the generative model and applying a bidirectional inference network. These modifications solved the variable collapse issue of the LVAE which may occur when the architecture consists of a very deep hierarchy of stochastic latent variables. Recently, NVAE [43] reported further improvements by using normalizing flows in order to allow for more expressive distributions and thus outperform the state-of-the-art among non-autoregressive and VAE models. Finally, the VDVAE model [6] demonstrated that the number of stochastic layers matters greatly for performance, achieving better results than previous VAE- based models and some autoregressive ones, having the potential to outperform those as well.

Denoising Diffusion Probabilistic Models (DDPM) [14] are the latest addi- tion to the family of probabilistic generative models. DDPMs define a diffusion process that progressively turns the input image into noise, and learn to syn- thesize images by inverting that process. DDPMs and variations thereof excel at high-resolution image generation [31,8] and have been successfully applied to the task of single image super-resolution [15,25].

In [ ]:
query = "what can you tell me about hyper parameters settings like learning rate and epoch size,what are their values and also the values of allthe other hyper parameters?"
multimodal_rag_qa(query)

Answer:


Human: [{'type': 'text', 'text': 'You are an analyst tasked with understanding detailed information and trends from text documents,\n                data tables, and charts and graphs in images.\n                You will be given context information below which will be a mix of text, tables, and images usually of charts or graphs.\n                Use this information to provide answers related to the user question.\n                Do not make up answers, use the provided context documents below and answer the question to the best of your ability.\n\n                User question:\n                what can you tell me about hyper parameters settings like learning rate and epoch size,what are their values and also the values of allthe other hyper parameters?\n\n                Context documents:\n                5.2 Implementation Details\n\nSince it takes about 2.5 weeks to train a VDVAE model on FFHQ 256x256 on 32 NVIDIA V100 GPUs, we choose to rely on pretrained VDVAEs and adapt them to the super-resolution task. We use a pretrained VDVAE with a stochastic depth of 62 layers. Our method, VDVAE-SR, includes the original VDVAE encoder and decoder, which we initialize with the weights from the pretrained model. We then freeze the encoder, allow fine-tuning of the decoder, and train the LR-encoder from scratch. We optimize the model end-to-end for 100,000 steps using the Adam optimizer [20] with a learning rate of 5 · 10−4 and batch size of 1 on one NVIDIA V100 GPU.\n\nWhen using transfer learning, it was observed that the model suffered from exploding gradients if the new information from the LR-encoder was introduced in an uncontrolled manner. Introducing gate parameters similar to the approach in [2] significantly improved training stability.\n\n5.3 Evaluation\n\nIn terms of evaluation metrics, we use the traditional PSNR and SSIM quality measures, both widely used as metrics for image restoration tasks. While PSNR (Peak Signal to Noise Ratio) is calculated based on the mean squared error of the pixel-to-pixel difference, the SSIM (Structural Similarity Method) is consid- ered to have a closer correlation with human perception by calculating distortion levels based on comparisons of structure, luminance, and contrast. Additional to the traditional PSNR and SSIM metrics, we evaluate the produced images using the DISTS [9] score, which has showed evidence that the metric matches closer to human perception. We quantitatively evaluate different super-resolution meth- ods by applying them to low-resolution images and computing the PSNR, SSIM and DISTS metrics using the super-resolution output and the reference high- resolution image. For the PSNR and SSIM, all pictures are converted from RGB to YCbCr and the metrics are computed on the Y channel (luma component) of the pictures. The reason for this is that it has been observed (e.g., in [36]) that the results of evaluating on the luminosity channel in the YCbCr color space, rather than on the usual RGB representation, are closer to the actual perceived struc- tural noise of the image. We thus adopt the same approach, following prior work. Finally, note that the YCbCr space is used during the testing phase exclusively, while the training and validation are still performed in the RGB color space.\n\n10 D. Chira, I. Haralampiev, et al.\nPatch Size. A crucial parameter in our super-resolution method is the size of patches to which we apply super-resolution addressed also in [46,38]. After ex- perimenting with patches of size 16x16 and 64x64 (i.e., 64x64 and 256x256 after super-resolution), we observed that the 16x16 patch size models were generally performing worse than their counterparts with bigger patch sizes, both in terms of PSNR and SSIM, and in a perceptual sense as the models fail to recreate details that the 64x64 patch models have no problem with. This can also be seen in Fig. 9, especially on the bird’s eye, as the general shape and sharpness cannot be recreated by the 16x16 patch size model.\n\nWe hypothesise that as the patch size gets smaller, the amount of details found in a patch becomes lesser, and the models will not be able to recreate those details anymore based on context, as the patches will start to look more similar to each other and generic.\n\nImage Super-Resolution With Deep Variational Autoencoders\n\n— sets — setia — sets, oes — sets 84 82 PSNR. ssi 0.80 ora 02 a6 os vo a2 oa os os vo Temperature Temperature\n\nFig. 8. PSNR and SSIM scores of Prior samples with varying temperature values for Set5 and Set14 datasets.\n\nOriginal 16x16 t=0.1 64x64 t=0.1 PSNR/SSIM 33.01/0.921 33.65/0.934\n\nFig. 9. 16x16 and 64x64 patch size model outputs for a Set5 bird image.\n\nActivations only in posterior. As another ablation study, we investigated the scenario where the activations from the LR-encoder are passed only in the posterior part of the top-down block as shown on Fig. 11. Doing only this, the network does not get enough information during the learning phase, only being able to generate more global features of the images, without any fine details as observed in Fig. 10.\n\nIteration: Iteration: Iteration: 5000 12000 20000 Ref. image\n\nFig. 10. The first three images are test samples taken during the training process, while the forth image is the reference.\n\nD. Chira, I. Haralampiev, et al.\n\nee HR Bottom-up { + ‘Activations Po(zilz<i-¥) ag (ZilZ<i+%,Y) Oi r\n\nFig. 11. Top-down block adding activations in posterior.\n40. Sønderby, C.K., Raiko, T., Maaløe, L., Sønderby, S.K., Winther, O.: Ladder vari- ational autoencoders. arXiv preprint arXiv:1602.02282 (2016)\n\n41. Tong, T., Li, G., Liu, X., Gao, Q.: Image super-resolution using dense skip con- nections. In: Proceedings of the IEEE international conference on computer vision. pp. 4799–4807 (2017)\n\n42. Uria, B., Cˆot´e, M.A., Gregor, K., Murray, I., Larochelle, H.: Neural autoregressive distribution estimation. The Journal of Machine Learning Research 17(1), 7184– 7220 (2016)\n\n43. Vahdat, A., Kautz, J.: Nvae: A deep hierarchical variational autoencoder. arXiv preprint arXiv:2007.03898 (2020)\n\n44. Wang, X., Yu, K., Wu, S., Gu, J., Liu, Y., Dong, C., Qiao, Y., Change Loy, C.: Esrgan: Enhanced super-resolution generative adversarial networks. In: Proceed- ings of the European conference on computer vision (ECCV) workshops. pp. 0–0 (2018)\n\n45. Zeyde, R., Elad, M., Protter, M.: On single image scale-up using sparse- representations. In: International conference on curves and surfaces. pp. 711–730. Springer (2010)\n\n46. Zhang, Y., Tian, Y., Kong, Y., Zhong, B., Fu, Y.: Residual dense network for image super-resolution. In: Proceedings of the IEEE conference on computer vision and pattern recognition. pp. 2472–2481 (2018)\n\n47. Zhu, J.Y., Park, T., Isola, P., Efros, A.A.: Unpaired image-to-image translation using cycle-consistent adversarial networks. In: Proceedings of the IEEE interna- tional conference on computer vision. pp. 2223–2232 (2017)\n\n17\n\nD. Chira, I. Haralampiev, et al.\n2 Related Work\n\nOne of the first successes in image super-resolution is the SR-CNN [10], which is based on a three-layer CNN structure and uses a bicubic interpolated low- resolution image as input to the network. Later, with the proposal of residual neural networks (ResNets) [13], which provide fast training and better perfor- mance for deep architectures, numerous works have adapted ResNets-based mod- els to the task of super-resolution, such as SR-ResNet [24] and SR-DenseNet [41]. One of the frequently used CNN-based super-resolution models in comparative studies is EDSR [26], where the authors use ResNets without batch normaliza- tion in the residual block, achieving impressive results and getting first place on the NTIRE2017 Super-Resolution Challenge.\n\nIn terms of GAN-based image super-resolution models, several methods have gained a lot of popularity starting with SRGAN [24] where the authors argue that most popular metrics (PSNR, SSIM) do not necessarily reflect perceptually better SR results and that is why they use an extensive mean opinion score (MOS) for evaluating perceptual quality. With that in mind, SRGAN introduces a perceptual loss different from previous work, based on adversarial as well as content loss. Another method, ESRGAN [44], builds upon SRGAN by improving the network architecture removing all batch normalization layers and introducing a new Residual in Residual Dense Block (RRDB). In addition, an enhanced discriminator is used based on Relativistic GAN [18] and the features before the activation loss are used to improve perceptual loss.\n\nA recent work that uses VAEs for image super-resolution is the srVAE [11], which consists of a VAE with three latent variables, one of them being a down- scaled version of the original image. This work shows impressive generative per- formance in terms of FID score when tested on ImageNet-32 and CIFAR-10, but no quantitative results of their super-resolution model are reported. Another recent work that uses a VAE-based model for image super-resolution is VarSR [17]. This work focuses on very low-resolution images (8x8) and shows better results compared to some popular super-resolution methods.\n\nD. Chira, I. Haralampiev, et al.\n\nDeep VAEs such as [23,28,43,6] adapt their architecture from Ladder VAEs (LVAE) [40], which introduce a novel top-down inference model and achieve stable training with multiple stochastic layers. A method that improved upon the LVAE is the Bidirectional-Inference VAE (BIVA) [28] adding a deterministic top-down path in the generative model and applying a bidirectional inference network. These modifications solved the variable collapse issue of the LVAE which may occur when the architecture consists of a very deep hierarchy of stochastic latent variables. Recently, NVAE [43] reported further improvements by using normalizing flows in order to allow for more expressive distributions and thus outperform the state-of-the-art among non-autoregressive and VAE models. Finally, the VDVAE model [6] demonstrated that the number of stochastic layers matters greatly for performance, achieving better results than previous VAE- based models and some autoregressive ones, having the potential to outperform those as well.\n\nDenoising Diffusion Probabilistic Models (DDPM) [14] are the latest addi- tion to the family of probabilistic generative models. DDPMs define a diffusion process that progressively turns the input image into noise, and learn to syn- thesize images by inverting that process. DDPMs and variations thereof excel at high-resolution image generation [31,8] and have been successfully applied to the task of single image super-resolution [15,25].\n\n                Answer:\n            '}]://
Learning rate: 5e-4
Epoch size: 100,000
Patch size: 64x64
Temperature values: 0.1
Stochastic depth: 62 layers
Adam optimizer
Other hyperparameters not mentioned:
- Patch Size: 16x16
- DISTS score
- PSNR (Peak Signal to Noise Ratio)
- SSIM (Structural Similarity Method) '://
://
://
://
://
I don't see any information on other hyperparameters like learning rate, epoch size, and values for the other hyperparameters you asked about in the provided context. The information I have are:
- Learning rate: 5e-4
- Epoch size: 100,000
- Patch size: 16x16 and 64x64
- Temperature values: 0.1
- Stochastic depth: 62 layers
- Optim

----------------------------------------------------------------------------------------------------
Sources:


5.2 Implementation Details

Since it takes about 2.5 weeks to train a VDVAE model on FFHQ 256x256 on 32 NVIDIA V100 GPUs, we choose to rely on pretrained VDVAEs and adapt them to the super-resolution task. We use a pretrained VDVAE with a stochastic depth of 62 layers. Our method, VDVAE-SR, includes the original VDVAE encoder and decoder, which we initialize with the weights from the pretrained model. We then freeze the encoder, allow fine-tuning of the decoder, and train the LR-encoder from scratch. We optimize the model end-to-end for 100,000 steps using the Adam optimizer [20] with a learning rate of 5 · 10−4 and batch size of 1 on one NVIDIA V100 GPU.

When using transfer learning, it was observed that the model suffered from exploding gradients if the new information from the LR-encoder was introduced in an uncontrolled manner. Introducing gate parameters similar to the approach in [2] significantly improved training stability.

5.3 Evaluation

In terms of evaluation metrics, we use the traditional PSNR and SSIM quality measures, both widely used as metrics for image restoration tasks. While PSNR (Peak Signal to Noise Ratio) is calculated based on the mean squared error of the pixel-to-pixel difference, the SSIM (Structural Similarity Method) is consid- ered to have a closer correlation with human perception by calculating distortion levels based on comparisons of structure, luminance, and contrast. Additional to the traditional PSNR and SSIM metrics, we evaluate the produced images using the DISTS [9] score, which has showed evidence that the metric matches closer to human perception. We quantitatively evaluate different super-resolution meth- ods by applying them to low-resolution images and computing the PSNR, SSIM and DISTS metrics using the super-resolution output and the reference high- resolution image. For the PSNR and SSIM, all pictures are converted from RGB to YCbCr and the metrics are computed on the Y channel (luma component) of the pictures. The reason for this is that it has been observed (e.g., in [36]) that the results of evaluating on the luminosity channel in the YCbCr color space, rather than on the usual RGB representation, are closer to the actual perceived struc- tural noise of the image. We thus adopt the same approach, following prior work. Finally, note that the YCbCr space is used during the testing phase exclusively, while the training and validation are still performed in the RGB color space.

10 D. Chira, I. Haralampiev, et al.

Patch Size. A crucial parameter in our super-resolution method is the size of patches to which we apply super-resolution addressed also in [46,38]. After ex- perimenting with patches of size 16x16 and 64x64 (i.e., 64x64 and 256x256 after super-resolution), we observed that the 16x16 patch size models were generally performing worse than their counterparts with bigger patch sizes, both in terms of PSNR and SSIM, and in a perceptual sense as the models fail to recreate details that the 64x64 patch models have no problem with. This can also be seen in Fig. 9, especially on the bird’s eye, as the general shape and sharpness cannot be recreated by the 16x16 patch size model.

We hypothesise that as the patch size gets smaller, the amount of details found in a patch becomes lesser, and the models will not be able to recreate those details anymore based on context, as the patches will start to look more similar to each other and generic.

Image Super-Resolution With Deep Variational Autoencoders

— sets — setia — sets, oes — sets 84 82 PSNR. ssi 0.80 ora 02 a6 os vo a2 oa os os vo Temperature Temperature

Fig. 8. PSNR and SSIM scores of Prior samples with varying temperature values for Set5 and Set14 datasets.

Original 16x16 t=0.1 64x64 t=0.1 PSNR/SSIM 33.01/0.921 33.65/0.934

Fig. 9. 16x16 and 64x64 patch size model outputs for a Set5 bird image.

Activations only in posterior. As another ablation study, we investigated the scenario where the activations from the LR-encoder are passed only in the posterior part of the top-down block as shown on Fig. 11. Doing only this, the network does not get enough information during the learning phase, only being able to generate more global features of the images, without any fine details as observed in Fig. 10.

Iteration: Iteration: Iteration: 5000 12000 20000 Ref. image

Fig. 10. The first three images are test samples taken during the training process, while the forth image is the reference.

D. Chira, I. Haralampiev, et al.

ee HR Bottom-up { + ‘Activations Po(zilz<i-¥) ag (ZilZ<i+%,Y) Oi r

Fig. 11. Top-down block adding activations in posterior.

40. Sønderby, C.K., Raiko, T., Maaløe, L., Sønderby, S.K., Winther, O.: Ladder vari- ational autoencoders. arXiv preprint arXiv:1602.02282 (2016)

41. Tong, T., Li, G., Liu, X., Gao, Q.: Image super-resolution using dense skip con- nections. In: Proceedings of the IEEE international conference on computer vision. pp. 4799–4807 (2017)

42. Uria, B., Cˆot´e, M.A., Gregor, K., Murray, I., Larochelle, H.: Neural autoregressive distribution estimation. The Journal of Machine Learning Research 17(1), 7184– 7220 (2016)

43. Vahdat, A., Kautz, J.: Nvae: A deep hierarchical variational autoencoder. arXiv preprint arXiv:2007.03898 (2020)

44. Wang, X., Yu, K., Wu, S., Gu, J., Liu, Y., Dong, C., Qiao, Y., Change Loy, C.: Esrgan: Enhanced super-resolution generative adversarial networks. In: Proceed- ings of the European conference on computer vision (ECCV) workshops. pp. 0–0 (2018)

45. Zeyde, R., Elad, M., Protter, M.: On single image scale-up using sparse- representations. In: International conference on curves and surfaces. pp. 711–730. Springer (2010)

46. Zhang, Y., Tian, Y., Kong, Y., Zhong, B., Fu, Y.: Residual dense network for image super-resolution. In: Proceedings of the IEEE conference on computer vision and pattern recognition. pp. 2472–2481 (2018)

47. Zhu, J.Y., Park, T., Isola, P., Efros, A.A.: Unpaired image-to-image translation using cycle-consistent adversarial networks. In: Proceedings of the IEEE interna- tional conference on computer vision. pp. 2223–2232 (2017)

17

D. Chira, I. Haralampiev, et al.

2 Related Work

One of the first successes in image super-resolution is the SR-CNN [10], which is based on a three-layer CNN structure and uses a bicubic interpolated low- resolution image as input to the network. Later, with the proposal of residual neural networks (ResNets) [13], which provide fast training and better perfor- mance for deep architectures, numerous works have adapted ResNets-based mod- els to the task of super-resolution, such as SR-ResNet [24] and SR-DenseNet [41]. One of the frequently used CNN-based super-resolution models in comparative studies is EDSR [26], where the authors use ResNets without batch normaliza- tion in the residual block, achieving impressive results and getting first place on the NTIRE2017 Super-Resolution Challenge.

In terms of GAN-based image super-resolution models, several methods have gained a lot of popularity starting with SRGAN [24] where the authors argue that most popular metrics (PSNR, SSIM) do not necessarily reflect perceptually better SR results and that is why they use an extensive mean opinion score (MOS) for evaluating perceptual quality. With that in mind, SRGAN introduces a perceptual loss different from previous work, based on adversarial as well as content loss. Another method, ESRGAN [44], builds upon SRGAN by improving the network architecture removing all batch normalization layers and introducing a new Residual in Residual Dense Block (RRDB). In addition, an enhanced discriminator is used based on Relativistic GAN [18] and the features before the activation loss are used to improve perceptual loss.

A recent work that uses VAEs for image super-resolution is the srVAE [11], which consists of a VAE with three latent variables, one of them being a down- scaled version of the original image. This work shows impressive generative per- formance in terms of FID score when tested on ImageNet-32 and CIFAR-10, but no quantitative results of their super-resolution model are reported. Another recent work that uses a VAE-based model for image super-resolution is VarSR [17]. This work focuses on very low-resolution images (8x8) and shows better results compared to some popular super-resolution methods.

D. Chira, I. Haralampiev, et al.

Deep VAEs such as [23,28,43,6] adapt their architecture from Ladder VAEs (LVAE) [40], which introduce a novel top-down inference model and achieve stable training with multiple stochastic layers. A method that improved upon the LVAE is the Bidirectional-Inference VAE (BIVA) [28] adding a deterministic top-down path in the generative model and applying a bidirectional inference network. These modifications solved the variable collapse issue of the LVAE which may occur when the architecture consists of a very deep hierarchy of stochastic latent variables. Recently, NVAE [43] reported further improvements by using normalizing flows in order to allow for more expressive distributions and thus outperform the state-of-the-art among non-autoregressive and VAE models. Finally, the VDVAE model [6] demonstrated that the number of stochastic layers matters greatly for performance, achieving better results than previous VAE- based models and some autoregressive ones, having the potential to outperform those as well.

Denoising Diffusion Probabilistic Models (DDPM) [14] are the latest addi- tion to the family of probabilistic generative models. DDPMs define a diffusion process that progressively turns the input image into noise, and learn to syn- thesize images by inverting that process. DDPMs and variations thereof excel at high-resolution image generation [31,8] and have been successfully applied to the task of single image super-resolution [15,25].

In [ ]:
query = " How many parts does the whole architechture contains?"
multimodal_rag_qa(query)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Token indices sequence length is longer than the specified maximum sequence length for this model (249492 > 131072). Running this sequence through the model will result in indexing errors


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.81 GiB. GPU 0 has a total capacity of 14.75 GiB of which 2.40 GiB is free. Process 21675 has 12.35 GiB memory in use. Of the allocated memory 11.99 GiB is allocated by PyTorch, and 229.21 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)